In [12]:
%%capture

!pip install ripser
!pip install persim
!pip install pyshp
!pip install kde-gpu
!pip install pyLDAvis==2.1.2
!pip install pandas
!pip install nltk
!pip install gensim

!pip install spacy
# !pip install pyLDAvis --upgrade

!python -m spacy download en_core_web_sm

%matplotlib inline

In [18]:
!pip3 install pickle5

     |████████████████████████████████| 132 kB 5.0 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219288 sha256=c6c86b5677480166666e247ce0485bc48c07018ade6657f56363dba7d2866d5b
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [6]:
import os
os.getcwd()

'/content'

In [9]:
os.chdir('/content/drive/Shareddrives/taxi_trends/Experiments/Twitter Data/Topic Modeling')

In [13]:
import topic_model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


In [14]:
import pyLDAvis

In [15]:
import gensim

In [16]:
import pandas as pd

In [19]:
# Load saved data
import pickle5 as pickle
with open('data_lemmatized_octfinal31102021175154.pkl', 'rb') as inp:
    data_lemmatized = pickle.load(inp)

In [20]:
twitter_df = pd.read_csv("twitter_processed_zonefiltered.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
# to be fixed; convert column to datetime
twitter_df["created_at"] = pd.to_datetime(twitter_df["created_at"])

In [22]:
twitter_df["created_at_dt"] = twitter_df["created_at"].dt.date

In [23]:
index = twitter_df.index

In [31]:
analysis_day = '2018-10-12'
zone = "East Village"
fil_indices = index[(twitter_df["created_at_dt"].astype("str") == analysis_day) & (twitter_df["taxizone"] == zone)].tolist()

In [32]:
data_lemmatized_fil = [data_lemmatized[i] for i in fil_indices]
len(data_lemmatized_fil)

127

In [33]:
import gensim.corpora as corpora# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [36]:
num_topics = 5

In [37]:
print("Building LDA model")
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# id2word, corpus, lda_model = topic_model.buildLDA(data_lemmatized, k=10)

Building LDA model


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

KeyboardInterrupt: ignored

In [ ]:
# lda_model.save('LDA_snapshots/oct_19')

In [ ]:
# Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
lda_model = gensim.models.lda_models.LDA.load("LDA_snapshots/2018_10")

print("Topics distribution")
doc_topic_dists = topic_model.get_doc_topic_dists(lda_model)
doc_lengths = [len(text) for text in data_lemmatized]

lda_model.print_topics()

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis